#SIGN LANGUAGE CHARACTER RECOGNITION CODE 
#Sign language is the only mode of communication for the hard-of-hearing community and thus it becomes important to develop a system that can recognizr sign language to bridge the gap of communication. 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as npy # linear algebra
import pandas as pds # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for m_filename in filenames:
        print(os.path.join(dirname, m_filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pds
import numpy as npy
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau

# Loading the ASL dataset

In [ ]:
#Sign Language MNIST” and is a public-domain free-to-use dataset with pixel information for around 1,000 images of each of 24 ASL Letters, excluding J and Z as they are gesture-based signs. Training data= (27,455 cases) and test data= (7172 cases)

#The dataset is preprocessed by converting the raw sign language video data into a sequence of frames. Each frame is preprocessed to extract features using a convolutional neural network (CNN). CNN is designed to learn and extract relevant features from sign language character images


In [ ]:
train_df = pds.read_csv (r"C:\Users\simra\Downloads\TEAM ALPHA\signmnisttrain\signmnisttrain.csv") 
test_df = pds.read_csv(r"C:\Users\simra\Downloads\TEAM ALPHA\sign_mnist_test\sign_mnist_test.csv")

In [ ]:
test = pds.read_csv(r"C:\Users\simra\Downloads\TEAM ALPHA\sign_mnist_test\sign_mnist_test.csv")
y = test['label']

In [ ]:

train_df.head()

# Data Visualization and Preprocessing¶

In [ ]:
plt.figure(figsize = (10,10)) # Label Count
sns.set_style("darkgrid")
sns.countplot(train_df['label'])

In [ ]:
y_train = train_df['label']
y_test = test_df['label']
del train_df['label']
del test_df['label']

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_b = LabelBinarizer()
y_train = label_b.fit_transform(y_train)
y_test = label_b.fit_transform(y_test)

In [ ]:
# Normalize the data
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
# Reshaping the data from 1-D to 3-D as required through input by CNN's
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [ ]:
f, ax = plt.subplots(2,5) 
f.set_size_inches(10, 10)
k = 0
for i in range(2):
    for j in range(5):
        ax[i,j].imshow(x_train[k].reshape(28, 28) , cmap = "gray")
        k += 1
    plt.tight_layout()    

# Data Augmentation

In [ ]:
# data augmentation to prevent overfitting

data_g = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


data_g.fit(x_train)

# Training The Model

In [ ]:

learning_rate_reduc = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

In [ ]:
model = Sequential()
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 24 , activation = 'softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

In [ ]:
history = model.fit(data_g.flow(x_train,y_train, batch_size = 128) ,epochs = 22 , validation_data = (x_test, y_test) , callbacks = [learning_rate_reduc])

In [ ]:
print("Accuracy of the model is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

In [ ]:
#Analysis after Model Training¶

In [ ]:
epochs = [i for i in range(20)]
fig , ax = plt.subplots(1,2)
train_accur = history.history['accuracy']
train_lossess = history.history['loss']
val_accur = history.history['val_accuracy']
val_lossess = history.history['val_loss']
fig.set_size_inches(16,9)

ax[0].plot(epochs , train_accur , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_accur , 'ro-' , label = 'Testing Accuracy')
ax[0].set_title('Training & Validation Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_lossess , 'g-o' , label = 'Training Loss')
ax[1].plot(epochs , val_lossess , 'r-o' , label = 'Testing Loss')
ax[1].set_title('Testing Accuracy & Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
plt.show()

In [ ]:
predictions_made = model.predict(x_test)
class_xx=npy.argmax(predictions_made, axis=1)
for i in range(len(class_xx)):
    if(class_xx[i] >= 9):
        class_xx[i]+=1
class_xx[:5]        

In [ ]:
classes_s = ["Class " + str(i) for i in range(25) if i != 9]
print(classification_report(y, predictions_made, target_names = classes_s))

In [ ]:
cmatrix  = confusion_matrix(y,predictions_made)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_sc = accuracy_score(y_test, predictions_made)
print('Accuracy Score = ', accuracy_sc)

In [ ]:
class_names_given = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I','K', 'L', 'M', 
               'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y' ]


In [ ]:
import random
vector = npy.vectorize(np.int_)
i = random.randint(1,len(predictions_made))
plt.imshow(x_test[i,:,:,0]) 
print("Predicted Label: ", class_names_given[int(predictions_made[i])])
print("True Label: ", class_names_given[int(y_test[i])])

In [ ]:
cmatrix = pds.DataFrame(cmatrix , index = [i for i in range(25) if i != 9] , columns = [i for i in range(25) if i != 9])

In [ ]:
plt.figure(figsize = (15,15))
sns.heatmap(cmatrix,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='')

In [ ]:
correct = npy.nonzero(predictions_made == y)[0]